In [12]:
%%capture --no-stderr
!pip install -r requirements.txt

In [13]:
import openai
import os
import pandas as pd
from typing import TypedDict, List, Dict, Any

from dotenv import load_dotenv

# Загружаем переменные окружения
load_dotenv()

True

In [14]:
# API ключ от OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Загружаем логи
df = pd.read_csv('logs/server_logs.csv')

### Наивное решение

In [15]:
# Функция для создания промпта
def create_prompt(row):
    prompt = f"""
Ты AIOps-ассистент. Анализируй метрики виртуальных серверов и определяй, является ли состояние сервера "Нормальным" или "Аномальным". Аномалией считается: CPU > 90%, использование RAM > 90%, свободное место на диске C: < 10%.

Примеры:
- Сервер: Web-Srv-01, CPU: 25%, RAM: 8/16 ГБ, Диск C: 75/100 ГБ -> Ответ: Нормальное
- Сервер: DB-Srv-02, CPU: 95%, RAM: 58/64 ГБ, Диск C: 185/200 ГБ -> Ответ: Аномальное (CPU 95%)

Теперь проанализируй:
Сервер: {row['server_name']}, CPU: {row['cpu_%']}%, RAM: {row['ram_used_gb']}/{row['ram_total_gb']} ГБ, Диск C: {row['c_disk_free_gb']}/{row['c_disk_total_gb']} ГБ
Ответ:
"""
    return prompt

In [16]:
# Анализируем первую запись для примера
first_row = df.iloc[0]
prompt_for_analysis = create_prompt(first_row)

NameError: name 'df' is not defined

In [17]:
# Отправляем запрос в OpenAI
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Ты помогаешь сетевому инженеру анализировать логи серверов."},
        {"role": "user", "content": prompt_for_analysis}
    ],
    temperature=0  # Для детерминированности ответов
)

NameError: name 'prompt_for_analysis' is not defined

In [ ]:
print("Данные:", f"Сервер {first_row['server_name']}, CPU: {first_row['cpu_%']}%")
print("Анализ ИИ:", response.choices[0].message['content'])

### Метрики для определения аномалий

Пример вычисляемых метрик для аномалий:
```
metrics = {
    'cpu_anomaly': cpu_usage > 90,                    # CPU > 90%
    'ram_anomaly': (ram_used / ram_total) > 0.9,      # RAM usage > 90%
    'disk_anomaly': (disk_free / disk_total) < 0.1,   # Free space < 10%
    'network_anomaly': network_usage > threshold,     # Если есть метрики сети
}
```

Вычисление производных (rate of change) (Временные метрики, тренды и изменения)
```
time_based_metrics = {
    'cpu_spike': current_cpu > (previous_cpu * 2),           # Резкий скачок CPU
    'ram_growth_rate': (current_ram - previous_ram) / time_delta,
    'disk_fill_rate': (previous_free - current_free) / time_delta,
    'sustained_high_load': cpu_usage > 80 for 5+ minutes,    # Длительная высокая нагрузка
}
```

Отклонение от нормального поведения
```
statistical_metrics = {
    'cpu_z_score': abs(current_cpu - mean_cpu_7d) / std_cpu_7d,
    'ram_deviation': current_ram > (mean_ram + 2 * std_ram),
    'unusual_pattern': night_hours & high_cpu,  # Высокий CPU ночью для офисного сервера
}
```